In [ ]:
def get_candidate_word_probabilities(input_text, candidate_words):
    tokenized_text = tokenizer.tokenize(input_text)
    masked_word_index = tokenized_text.index('[MASK]')
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze_(0)

    with torch.no_grad():
        output = model(input_ids)

    predictions = output.logits[0, masked_word_index].softmax(dim=0)

    candidate_probabilities = {}
    word_idss = []
    probss = []
    avg_probs = []
    for word in candidate_words:
        # Tokenize the candidate word
        tokenized_word = tokenizer.tokenize(word)

        # Convert the tokens to ids
        word_ids = tokenizer.convert_tokens_to_ids(tokenized_word)
        word_idss.append(word_ids)
        
        sum = 0
        for word_id in word_ids:
            prediction_w = predictions[word_id].item()
            probss.append(prediction_w)
            sum = sum + prediction_w
            
        # Calculate the average probability of the tokens
        avg_prob = sum / len(word_ids)
        avg_probs.append(avg_prob)
        candidate_probabilities[word] = avg_prob
        
    print(word_idss)
    idtowordIDS = [tokenizer.convert_ids_to_tokens(word_id) for word_id in word_idss]
    print(idtowordIDS)
    print(probss)
    print(avg_probs)
    """
    # Tokenize and convert all candidate words to ids at once

    tokenized_candidate_words_uc = [tokenizer.tokenize(word)for word in candidate_words]
    print(tokenized_candidate_words_uc)
    
    
   
    tokenized_candidate_words = ["".join(word) for word in tokenized_candidate_words_uc]
    print(tokenized_candidate_words)
    
    candidate_word_ids = [tokenizer.convert_tokens_to_ids(TCW) for TCW in tokenized_candidate_words]
    print(candidate_word_ids)
    
    c_candidate_word_ids = [word_id for word_id in candidate_word_ids if word_id != 1]
    print(c_candidate_word_ids)
    
    t_txken = tokenized_candidate_words[0]
    print(t_txken)
    
    t_predict = predictions[tokenizer.convert_tokens_to_ids(t_txken)].item()
    print(t_predict)
    
    
    # Use a list comprehension to populate the dictionary
    candidate_probabilities = {word: predictions[word_id].item() for word, word_id in zip(candidate_words, candidate_word_ids)}
    """

    return candidate_probabilities


def generate_probabilties_end(example):
    input_text = example["Masked"]
    candidates = example["Options"]

            
    word_probabilities = get_candidate_word_probabilities(input_text, candidates)

    sorted_words = sorted(word_probabilities, key=word_probabilities.get, reverse=True)
    most_probable_word = sorted_words[0]

    return word_probabilities, sorted_words, most_probable_word